In [0]:
import re
from pyspark.sql import DataFrame

def to_snake_case(name):
    name = re.sub(r"([a-z0-9])([A-Z])", r"\1_\2", name)
    name = re.sub(r"([A-Z]+)([A-Z][a-z])", r"\1_\2", name)
    return name.lower()


def clean_and_snake_case_columns(df):
    cleaned_cols = []
    for col_name in df.columns:
        cleaned_name = re.sub(r"[ (){};\n\t=]", "", col_name).strip().replace(" ", "_")
        snake_case_name = to_snake_case(cleaned_name)
        cleaned_cols.append(snake_case_name)
    return df.toDF(*cleaned_cols)


def read_delta_with_snake_case(spark, path):
    df = spark.read.format("delta").load(path)
    return clean_and_snake_case_columns(df)


def get_store_product_data(product_df, store_df):
    return store_df.join(product_df, on="store_id", how="inner")


def enrich_sales_with_store_product(sales_df, store_product_df):
    return sales_df.join(store_product_df, on="product_id", how="inner")
